In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
df = pd.read_csv("../dataset/raw/dump.csv")
df.head(5)

,Unnamed: 0,ageEstimate,companyFollowerCount,companyHasLogo,companyName,companyStaffCount,companyUrl,companyUrn,connectionsCount,country,...,mbrLocationCode,mbrTitle,memberUrn,posLocation,posLocationCode,posTitle,positionId,startDate,avgMemberPosDuration,avgCompanyPosDuration
0,0,41.0,198859.0,5a92c12e-0a20-482b-95ec-e705a6294730-2.png,Commonwealth Bank,32905.0,http://www.commbank.com.au/,urn:li:company:2848,500.0,au,...,"urn:li:fs_region:(au,4910)",Portfolio Executive at Commonwealth Bank,urn:li:member:10013136,NaN,NaN,Portfolio Executive,240345.0,2014-07-01,760.5000,989.9361
1,1,41.0,198859.0,5a92c12e-0a20-482b-95ec-e705a6294730-2.png,Commonwealth Bank,32905.0,http://www.commbank.com.au/,urn:li:company:2848,500.0,au,...,"urn:li:fs_region:(au,4910)",Portfolio Executive at Commonwealth Bank,urn:li:member:10013136,"Sydney, Australia","urn:li:fs_region:(au,4910)",Solution Delivery Executive,240346.0,2013-11-01,760.5000,989.9361
2,2,41.0,10047.0,b1c3fd9a-8f21-48f8-8321-d773b1ff3cb7-2.png,CommSec,619.0,http://www.commsec.com.au,urn:li:company:2851,500.0,au,...,"urn:li:fs_region:(au,4910)",Portfolio Executive at Commonwealth Bank,urn:li:member:10013136,NaN,NaN,Project Manager,240348.0,2008-08-01,760.5000,747.2308
3,3,41.0,198859.0,5a92c12e-0a20-482b-95ec-e705a6294730-2.png,Commonwealth Bank,32905.0,http://www.commbank.com.au/,urn:li:company:2848,500.0,au,...,"urn:li:fs_region:(au,4910)",Portfolio Executive at Commonwealth Bank,urn:li:member:10013136,NaN,NaN,Project Manager,240350.0,2007-02-01,760.5000,989.9361
4,4,30.0,300723.0,0f982557-a907-409e-af20-1feb0e0813cd-2.png,PayPal,22522.0,http://www.paypal.com,urn:li:company:1482,500.0,au,...,"urn:li:fs_region:(au,4910)","Senior Marketing Manager, PayPal",urn:li:member:100186032,NaN,NaN,Senior Marketing Manager,229893.0,2017-01-01,395.2857,683.3496


In [3]:
df['text_raw'] = (
    df['posTitle'].fillna('') + ' ' +
    df['mbrTitle'].fillna('') + ' ' +
    df['companyName'].fillna('') + ' ' +
    df['country'].fillna('') + ' ' +
    df['posLocation'].fillna('')
)


In [4]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sadat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sadat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [6]:
def preprocess(text):
    # lowercase
    text = text.lower()
    
    # remove urls
    text = re.sub(r'http\S+|www\S+', ' ', text)
    
    # remove punctuation & numbers
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    
    # remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # remove stopwords & lemmatization
    words = text.split()
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    
    return " ".join(words)

In [7]:
df['text_clean'] = df['text_raw'].apply(preprocess)

In [8]:
df[['posTitle','mbrTitle','text_clean']].head(10)

,posTitle,mbrTitle,text_clean
0,Portfolio Executive,Portfolio Executive at Commonwealth Bank,portfolio executive portfolio executive common...
1,Solution Delivery Executive,Portfolio Executive at Commonwealth Bank,solution delivery executive portfolio executiv...
2,Project Manager,Portfolio Executive at Commonwealth Bank,project manager portfolio executive commonweal...
3,Project Manager,Portfolio Executive at Commonwealth Bank,project manager portfolio executive commonweal...
4,Senior Marketing Manager,"Senior Marketing Manager, PayPal",senior marketing manager senior marketing mana...
5,"Digital and Social Engagement Leader, Optus Bu...","Senior Marketing Manager, PayPal",digital social engagement leader optus busines...
6,Social Brand Marketing,"Senior Marketing Manager, PayPal",social brand marketing senior marketing manage...
7,Brand Strategist,"Senior Marketing Manager, PayPal",brand strategist senior marketing manager payp...
8,Brand Expression Lead,"Senior Marketing Manager, PayPal",brand expression lead senior marketing manager...
9,Brand System Graduate,"Senior Marketing Manager, PayPal",brand system graduate senior marketing manager...


In [9]:
df.to_csv('../dataset/cooked/preprocessed.csv', index=False)